In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GITHUB_API_KEY = os.getenv("GITHUB_API_KEY")

In [10]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.runners import InMemoryRunner, Runner
from google.adk.models.google_llm import Gemini
from google.adk.models.lite_llm import LiteLlm
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams, StreamableHTTPConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

In [3]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

### 2.1: How MCP Works

MCP connects your agent (the **client**) to external **MCP servers** that provide tools:

- **MCP Server**: Provides specific tools (like image generation, database access)
- **MCP Client**: Your agent that uses those tools
- **All servers work the same way** - standardized interface

**Architecture:**
```
┌──────────────────┐
│   Your Agent     │
│   (MCP Client)   │
└────────┬─────────┘
         │
         │ Standard MCP Protocol
         │
    ┌────┴────┬────────┬────────┐
    │         │        │        │
    ▼         ▼        ▼        ▼
┌────────┐ ┌─────┐ ┌──────┐ ┌─────┐
│ GitHub │ │Slack│ │ Maps │ │ ... │
│ Server │ │ MCP │ │ MCP  │ │     │
└────────┘ └─────┘ └──────┘ └─────┘
```

In [4]:
mcp_server_image = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",
            args=['-y',"@modelcontextprotocol/server-everything"],
            tool_filter = ["getTinyImage"]
        ),
        timeout=30
    )
)

In [5]:
# Create image agent with MCP integration
image_agent = LlmAgent(
    name = "image_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="Use the MCP Tool to generate images for user queries",
    tools=[mcp_server_image]
)

In [6]:
runner = InMemoryRunner(agent = image_agent)
response = await runner.run_debug("Provide a sample tiny image", verbose=True)


 ### Created new session: debug_session_id

User > Provide a sample tiny image


/home/poojan/programs/personal-programs/5dgai/.venv/lib/python3.11/site-packages/google/adk/tools/mcp_tool/mcp_tool.py:101: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


image_agent > [Calling tool: getTinyImage({})]
image_agent > [Tool result: {'content': [{'type': 'text', 'text': 'This is a tiny image:'}, {'type': 'image', 'data': 'iVBORw0KG...]
image_agent > This is a tiny image:
The image above is the MCP tiny image.


In [7]:
from IPython.display import display, Image as IPImage
import base64

for event in response:
    if event.content and event.content.parts:
        for part in event.content.parts:
            if hasattr(part, "function_response") and part.function_response:
                for item in part.function_response.response.get("content", []):
                    if item.get("type") == "image":
                        display(IPImage(data=base64.b64decode(item["data"])))

In [11]:
kaggle_mcp_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command='npx',
            args=[
                '-y',
                'mcp-remote',
                'https://www.kaggle.com/mcp'
            ],
        ),
        timeout=1000
    )
)

In [12]:
kaggle_agent = LlmAgent(
    name = "kaggle_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="Use the MCP Tool to search, download and perform multiple actions for database in kaggle",
    tools=[kaggle_mcp_server]
)

In [13]:
kaggle_runner = InMemoryRunner(agent=kaggle_agent)
response= await kaggle_runner.run_debug("I need large dataset for weapon detection in multiple scenerious", verbose = True)


 ### Created new session: debug_session_id

User > I need large dataset for weapon detection in multiple scenerious


/home/poojan/programs/personal-programs/5dgai/.venv/lib/python3.11/site-packages/google/adk/tools/mcp_tool/mcp_tool.py:101: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


kaggle_agent > [Calling tool: search_datasets({'request': {'search': 'weapon detection', 'size':...)]
kaggle_agent > [Tool result: {'content': [{'type': 'text', 'text': '{\n  "datasets": [\n    {\n      "id": 4895326,\n      "ref":...]


In [14]:
response

[Event(model_version='gemini-2.5-flash-lite', content=Content(
   parts=[
     Part(
       function_call=FunctionCall(
         args={
           'request': {
             'search': 'weapon detection',
             'size': 'Large'
           }
         },
         id='adk-b75baf9b-cda9-442b-be28-823ad4aa3f39',
         name='search_datasets'
       )
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   cache_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=13960
     ),
   ],
   cached_content_token_count=13960,
   candidates_token_count=26,
   prompt_token_count=14005,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=14005
     ),
   ],
   total_to

In [4]:
github_mcp_server = McpToolset(
    connection_params=StreamableHTTPConnectionParams(
        url="https://api.githubcopilot.com/mcp/",
        headers={
            "Authorization": f"Bearer {GITHUB_API_KEY}",
            "X-MCP-Toolsets": "all",
            "X-MCP-Readonly": "true"
        }
    )
)

In [5]:
github_agent = LlmAgent(
    name = "github_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="Use the MCP Tool to perform varioud github actions",
    tools=[github_mcp_server]
)

In [9]:
runner = InMemoryRunner(agent=github_agent)
response = await runner.run_debug("List files in https://github.com/poojan-solanki/bounding-boxes-by-gemini")


 ### Created new session: debug_session_id

User > List files in https://github.com/poojan-solanki/bounding-boxes-by-gemini


github_agent > The repository contains the following files:
- README.md
- bounding_boxes.py
- label_images.py
- test_code.py


#### Long-Running Operations (Human-in-the-Loop)

In [5]:
LARGE_ORDER_THRESHHOLD = 5

def place_shipping_order(num_containers: int, destination: str, tool_context: ToolContext):
    """Places a shipping order. Requires approval if ordering more than 5 containers (LARGE_ORDER_THRESHOLD).

    Args:
        num_containers: Number of containers to ship
        destination: Shipping destination

    Returns:
        Dictionary with order status
    """

    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 1: Small orders (≤5 containers) auto-approve
    if num_containers <= LARGE_ORDER_THRESHHOLD:
        return {
            "status": "approved",
            "order_id": f"ORD-{num_containers}-AUTO",
            "num_containers":num_containers,
            "destination": destination,
            "message": f"Order auto-appproved: {num_containers} containers at {destination}"
        }
    
    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 2: This is the first time this tool is called. Large orders need human approval - PAUSE here.
    if not tool_context.tool_confirmation:
        tool_context.request_confirmation(
            hint=f"⚠️ Large order: {num_containers} containers to {destination}. Do you want to approve?",
            payload={"num_ocntainers": num_containers, "destination": destination}
        )
        return {  # This is sent to agent
            "status" : "pending",
            "message": f"Order for {num_containers} containers requires approval",
        }
    
    # -----------------------------------------------------------------------------------------------
    # -----------------------------------------------------------------------------------------------
    # SCENARIO 3: The tool is called AGAIN and is now resuming. Handle approval response - RESUME here.
    if tool_context.tool_confirmation.confirmed:
        return {
            "status": "approved",
            "order_id": f"ORD-{num_containers}-HUMAN",
            "num_containers": num_containers,
            "destination": destination,
            "message": f"Order approved: {num_containers} containers to {destination}",
        }
    else:
        return {
            "status": "rejected",
            "message": f"Order rejected: {num_containers} containers to {destination}",
        }

### Understanding the Code

<img src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day2/lro-tool.png" width="1000" alt="Long-running operation tool">

In [8]:
shipping_agent = LlmAgent(
    name="shipping_agent",
    model=Gemini(model="gemini-2.5-flash", retry_options=retry_config),
    instruction="""You are a shipping coordinator assistant.
  
  When users request to ship containers:
   1. Use the place_shipping_order tool with the number of containers and destination
   2. If the order status is 'pending', inform the user that approval is required
   3. After receiving the final result, provide a clear summary including:
      - Order status (approved/rejected)
      - Order ID (if available)
      - Number of containers and destination
   4. Keep responses concise but informative
  """,
    tools=[FunctionTool(func=place_shipping_order)]
)

In [9]:
# Wrap the agent in a resuable app - THIS IS THE KEY FOR LONG RUNNING APPS!

shipping_app = App(
    name= "shipping_coordinator",
    root_agent=shipping_agent,
    resumability_config=ResumabilityConfig(is_resumable=True)
) 

/tmp/ipykernel_133512/133059918.py:6: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True)


In [11]:
session_service = InMemorySessionService()
shipping_runner = Runner(
    app=shipping_app,
    session_service=session_service
)